# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Построим модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке.

## Откроим и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Открыли и изучили файл, видим, что пропусков в данных нет, названия столбцов понятны, типы данных соответствующие.**

## Разобьем данные на выборки

**Разделили данные исходя из расчета 3:1:1 для обучающей, валидационной и тестовой выборок соответственно.**

In [5]:
df_train, df_temporary = train_test_split(df, test_size=0.4, random_state=12345)#Разделили исходные данные на обучающую, валидационную и тестовую выборки
df_valid, df_test = train_test_split(df_temporary, test_size=0.5, random_state=12345)

## Исследуем модели

In [6]:
#сохраним обучающие и валидационные признаки в переменные
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

**Сначала проверим модель DecisionTreeClassifier.**

In [7]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


**Исходя из оценки качества на валидационной выборке, видно что лучший результат показал параметр max_depth = 3.**

**Теперь проверим модель RandomForestClassifier.**

In [8]:
for est in range(1, 13):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    print("n_estimators =", est, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

n_estimators = 1 : 0.7107309486780715
n_estimators = 2 : 0.7636080870917574
n_estimators = 3 : 0.7387247278382582
n_estimators = 4 : 0.7713841368584758
n_estimators = 5 : 0.749611197511664
n_estimators = 6 : 0.7807153965785381
n_estimators = 7 : 0.7682737169517885
n_estimators = 8 : 0.7822706065318819
n_estimators = 9 : 0.7729393468118196
n_estimators = 10 : 0.7853810264385692
n_estimators = 11 : 0.7838258164852255
n_estimators = 12 : 0.7869362363919129


**Исходя из оценки качества на валидационной выборке, видно что лучший результат показал параметр n_estimators = 12.**

**Проверим также модель LogisticRegression.**

In [9]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print(result)

0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Исходя из результатов проверки, видим, что самый высокий accuracy у модели RandomForestClassifier , ее и будем проверять на тестовой выборке.**

## Проверим модель на тестовой выборке

**Так как в предыдущей главе мы выяснили, что у RandomForestClassifier самый высокий accuracy, то ее и проверим.**

In [10]:
#сохраним в переменные признаки для тестовой выборки
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [11]:
model = RandomForestClassifier(random_state=12345, n_estimators=12) 
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test)
print(accuracy_score(target_test, predictions_test))

0.7869362363919129


**Мы получили результат accuracy 0,78 это выше 0,75 как и требуется в задании.**

## Проверим модели на адекватность

**Сравним качество своей модели с качеством константной модели.**

In [12]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
model = DummyClassifier(constant=1, random_state=12345, strategy="constant")
model.fit(features_test, target_test)
predictions_test = model.predict(features_test)

In [13]:
model.score(features_test, target_test)

0.3157076205287714

In [14]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
model = DummyClassifier(constant=0, random_state=12345, strategy="constant")
model.fit(features_test, target_test)
predictions_test = model.predict(features_test)

In [15]:
model.score(features_test, target_test)

0.6842923794712286

**В обоих случаях accuracy константной модели на тестовой выборке - ниже чем качество модели RandomForestClassifier, наверно можно сделать вывод, что наша модель адекватна.**